In [1]:
!pip install dionysus 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for dionysus: filename=dionysus-2.0.10-cp310-cp310-linux_x86_64.whl size=413795 sha256=f001de0f6d78280f3f6d4e4f3b784113b3e367be09465af9c9bfd434c16d240a
  Stored in directory: /root/.cache/pip/wheels/1d/8f/2a/22f69fac8dab81069f0501bdd69961a69e82b8b3007f191a15
Successfully built dionysus


In [9]:
import numpy as np
import dionysus as d
from copy import deepcopy

In [7]:
def check_collision(current_circle, circles):
    """
        checks whether current_circle not intersects with circles
        returns: False if there is a collision, True otherwise
    """
    x, y, radius = current_circle
    if x-radius < 0 or x+radius > 1 or y-radius < 0 or y+radius > 1:
        return False
    return all(np.sqrt((x - xc)**2 + (y - yc)**2) >= (radius + rc) for xc, yc, rc in circles)

def in_circle(point, circle):
        """
            checks whether point is inside the circle
        """
        x, y = point
        xc, yc, r = circle
        return (x - xc) ** 2 + (y - yc) ** 2 < r ** 2


def generate_circles(num_circles,
                     r_boundaries,
                     x_boundaries,
                     y_boundaries,
                     num_attempts=1000,
                     seed=None):
    """
        Generates non-intersecting circles inside [0,1]x[0,1].
        num_circles - number of circles
        r_boundaries - circles radius boundaries
        x/y_boundaries - circles placement boundaries

        returns: list of non-intersecting circles
    """
    np.random.seed(seed)
    circles = []
    for _ in range(num_circles):
        for _ in range(num_attempts):
            radius = np.random.uniform(*(r_boundaries))
            x = np.random.uniform(x_boundaries[0] + radius, x_boundaries[1] - radius)
            y = np.random.uniform(y_boundaries[0] + radius, y_boundaries[1] - radius)
            new_circle = (x, y, radius)
            if check_collision((x, y, radius), circles):
                circles.append(new_circle)
                break
    return circles


def generate_example(num_points,
                     num_circles,
                     r_boundaries,
                     x_boundaries=(0, 1),
                     y_boundaries=(0, 1),
                     seed=None):
    """
    """
    np.random.seed(seed)
    points = np.random.rand(num_points, 2)
    circles = generate_circles(num_circles, r_boundaries, x_boundaries, y_boundaries, seed=seed)
    filtered_points = np.array([point for point in points if not any(in_circle(point, circle) for circle in circles)])
    return filtered_points, circles

In [8]:
X, _ = generate_example(num_circles=1, num_points=5, r_boundaries=(0.1, 0.15), x_boundaries=(0.5, 0.99))
print(len(X))
f = d.fill_rips(X, 2, 10)
f2 = d.Filtration()
for i, s in enumerate(f):
    print(s, list(s))
    f2.append(d.Simplex(list(s), i))
for s in f2:
    print(s)

5
<0> 0 [0]
<1> 0 [1]
<2> 0 [2]
<3> 0 [3]
<4> 0 [4]
<0,3> 0.0501644 [0, 3]
<3,4> 0.132758 [3, 4]
<0,4> 0.176822 [0, 4]
<0,3,4> 0.176822 [0, 3, 4]
<0,2> 0.436868 [0, 2]
<2,4> 0.452951 [2, 4]
<0,2,4> 0.452951 [0, 2, 4]
<2,3> 0.454946 [2, 3]
<0,2,3> 0.454946 [0, 2, 3]
<2,3,4> 0.454946 [2, 3, 4]
<1,2> 0.666308 [1, 2]
<1,4> 0.702514 [1, 4]
<1,2,4> 0.702514 [1, 2, 4]
<1,3> 0.819015 [1, 3]
<1,2,3> 0.819015 [1, 2, 3]
<1,3,4> 0.819015 [1, 3, 4]
<0,1> 0.845434 [0, 1]
<0,1,2> 0.845434 [0, 1, 2]
<0,1,3> 0.845434 [0, 1, 3]
<0,1,4> 0.845434 [0, 1, 4]
<0> 0
<1> 1
<2> 2
<3> 3
<4> 4
<0,3> 5
<3,4> 6
<0,4> 7
<0,3,4> 8
<0,2> 9
<2,4> 10
<0,2,4> 11
<2,3> 12
<0,2,3> 13
<2,3,4> 14
<1,2> 15
<1,4> 16
<1,2,4> 17
<1,3> 18
<1,2,3> 19
<1,3,4> 20
<0,1> 21
<0,1,2> 22
<0,1,3> 23
<0,1,4> 24


In [5]:
X = np.array(X)

In [7]:
res = np.inf
for p in X:
    dst = 0
    for p2 in X:
        dst = max(dst, np.linalg.norm(p - p2))
    res = min(res, dst)

In [8]:
res

0.6183030147837496

In [8]:
f = d.fill_rips(X, 2, res)
print(f)

KeyboardInterrupt: 

In [ ]:
%%time
f = d.fill_rips(X, 2, res)
m = d.homology_persistence(f)
print(len(m))

In [2]:
#from sys import orig_argv
import numpy as np
from scipy import sparse
import sys

from itertools import combinations, groupby
from bisect import bisect
from operator import attrgetter

from dataclasses import dataclass
from dataclasses import astuple, asdict

from typing import Tuple, Dict, List

@dataclass
class Simplex:
    vertices: Tuple[int]
    index: int = None
    time: float = None
    weight: float = None

    def __repr__(self):
        return "({})".format(", ".join(map(str, self.vertices)))

    @property
    def dim(self):
        return len(self.vertices) - 1

    @property
    def boundary(self):
        if self.dim==0:
            faces = []
        else:
            faces = [Simplex(item) for item in combinations(self.vertices, self.dim)][::-1]
        return faces

@dataclass
class Chain:
    elements: List[Simplex]
    coefficients: List[float]

@dataclass
class PersistenceRepresentative:
    birth_simplex: Simplex
    death_simplex: Simplex
    # birth_cycle: Chain = None
    # death_cycle: Chain = None
    birth_harmonic_cycle: np.ndarray
    death_harmonic_cycle: np.ndarray

@dataclass
class PersistenceDiagram:
    elements: List[PersistenceRepresentative]

    def num_representatives(self, dim=0):
        n_representatives = {0: 0, 1: 0}

        for representative in self.elements:
            representative_dim = representative.birth_simplex.dim
            n_representatives[representative_dim] = n_representatives[representative_dim] + 1

        return n_representatives[dim]

    def representatives_graded(self, k=0):

        representatives_graded = {}

        representatives = sorted(self.elements, key=lambda element: (element.birth_simplex.dim)) # , element.birth_simplex.index, element.death_simplex.index

        for k_repr, k_representatives in groupby(representatives, key=lambda representative: representative.birth_simplex.dim):
            k_representatives = list(k_representatives)
            representatives_graded[k_repr] = k_representatives

        return representatives_graded[k]

    def as_numpy(self, index=False):
        pd = np.zeros((len(self.elements), 3))
        
        sorted_elements = sorted(self.elements, key=lambda element: (element.birth_simplex.dim, element.birth_simplex.index, element.death_simplex.index))

        for i, element in enumerate(sorted_elements):
            if index==False:
                pd[i,:] = np.array([element.birth_simplex.dim, element.birth_simplex.time, element.death_simplex.time])
            else:
                pd[i,:] = np.array([element.birth_simplex.dim, element.birth_simplex.index, element.death_simplex.index])

        return pd.astype(int)

class FilteredComplex:
    # TODO:
    # boundary21, boundary10 (up to index/time, default=None means all matrix)
    # laplacian1 (up to index/time, default=None means all matrix)
    # 0- and 1-chain harmonic representatives

    def __init__(self, filtration: List[Simplex], dionf, oriented=False):
        self.filtration = filtration
        self.dionf = dionf
        self.oriented = oriented
        self.boundary_matrix = None
        self.oriented_boundary_matrix = None
        self.reduced_boundary_matrix = None
        self.persistence_diagram = None
        
        self.idionf = d.Filtration()
        for i, s in enumerate(self.dionf):
            self.idionf.append(d.Simplex(list(s), i))

        self.simplex_to_index = {}
        for simplex in self.filtration:
            self.simplex_to_index[simplex.vertices] = simplex.index

        n_simplices = len(self.filtration)
        self.n_simplices = n_simplices
        self.boundary_matrix = sparse.lil_matrix((n_simplices, n_simplices), dtype=int)
        self.oriented_boundary_matrix = sparse.lil_matrix((n_simplices, n_simplices), dtype=int)

        # building (oriented) boundary matrix
        for simplex in self.filtration:
            for q, face in enumerate(simplex.boundary):
                i, j = self.simplex_to_index[face.vertices], simplex.index
                self.boundary_matrix._set_intXint(i, j, 1)
                self.oriented_boundary_matrix._set_intXint(i, j, (-1)**q)
#                 self.boundary_matrix[i,j] = 1
#                 self.oriented_boundary_matrix[i,j] = (-1)**q # orientation
        print(self.boundary_matrix.shape, self.boundary_matrix.nnz)
        print("Done")
        sys.stdout.flush()

        # get graded filtration
        self.filtration_graded = {}
        filtration_index = sorted(self.filtration, key=lambda simplex: (len(simplex.vertices), simplex.index))
        for k, k_simplices in groupby(filtration_index, key=lambda simplex: len(simplex.vertices)):
            k_simplices = list(k_simplices)
            self.filtration_graded[k-1] = k_simplices

    def filtration_at_index(self, index=0, k=None):
        
        k_simplices_at_index = []

        if k is None:
            for simplex in self.filtration:
                if (simplex.index<=index):
                    k_simplices_at_index.append(simplex)
                else:
                    break
        
        else:
            for simplex in self.filtration_graded[k]:
                if (simplex.index<=index):
                    k_simplices_at_index.append(simplex)
                else:
                    break

        return k_simplices_at_index

    def get_reduced_boundary_matrix(self):
        
        def matrix_reduction(matrix):
            matrix = sparse.lil_matrix((self.n_simplices, self.n_simplices), dtype=int)
            m = d.homology_persistence(self.dionf)
            print("m calculated", type(matrix), len(m))
            self.dionm = m
            sys.stdout.flush()
            for i, c in enumerate(m):
                if c:
                    for t in c:
                        matrix._set_intXint(t.index, i, 1)
#                         matrix[t.index, i] = 1
            return matrix

        if (self.reduced_boundary_matrix==None): # cached
            print("!!!", type(self.boundary_matrix))
            self.reduced_boundary_matrix = matrix_reduction(self.boundary_matrix)
            print("to get_persistence_diagram")
            sys.stdout.flush()
            self.persistence_diagram = self.get_persistence_diagram()

        return self.reduced_boundary_matrix

    def view_boundary_matrix(self, index=None, order=1):
        
        self.simplices_at_index = {}
        self.simplices_index_idx = {}
        #print(len(self.filtration))

        filtration_index = sorted(self.filtration[:index+1], key=lambda simplex: (len(simplex.vertices), simplex.index))
        for k, k_simplices in groupby(filtration_index, key=lambda simplex: len(simplex.vertices)):
            k_simplices = list(k_simplices)
            self.simplices_at_index[k-1] = k_simplices
            self.simplices_index_idx[k-1] = [simplex.index for simplex in k_simplices]

        #print(self.oriented_boundary_matrix.shape, self.simplices_index_idx.keys())
        if order==1:
            B = self.oriented_boundary_matrix[self.simplices_index_idx[0],:][:,self.simplices_index_idx[1]]
        elif order==2:
            B = self.oriented_boundary_matrix[self.simplices_index_idx[1],:][:,self.simplices_index_idx[2]]

        return B

    def laplacian_matrix(self, index=None):
        B0 = self.view_boundary_matrix(index, order=1)
        B1 = self.view_boundary_matrix(index, order=2)
        L1 = B0.T @ B0 + B1 @ B1.T
        return L1

    def spectra(self, index=None):
        L1 = self.laplacian_matrix(index)
        print(type(L1), L1.shape)
        #sys.stdout.flush()
        ##eigenvalues, eigenvectors = np.linalg.eigh(L1.toarray())
        eigenvalues, eigenvectors = sparse.linalg.eigsh(L1.asfptype(), k=10, which='SM')#np.linalg.eigh(L1), sigma=0
        return eigenvalues, eigenvectors

    def get_persistence_diagram(self):
        dgms = d.init_diagrams(self.dionm, self.idionf)
        print(d.init_diagrams(self.dionm, self.dionf), d.init_diagrams(self.dionm, self.idionf))
        sys.stdout.flush()
        persistence_representatives = []
        for i, dgm in enumerate(dgms):
            for pt in dgm:
                if i == 1 and pt.death - pt.birth > 1:
                    print(i, pt.birth, pt.death)
                    birth_simplex, death_simplex = self.filtration[int(pt.birth)], self.filtration[int(pt.death)]
                    persistence_representative = PersistenceRepresentative(birth_simplex, death_simplex, np.zeros(1), np.zeros(1))
                    persistence_representatives.append(persistence_representative)
        return PersistenceDiagram(persistence_representatives)
        
        def low(column):
            column = (column!=0).astype(int)
            argwhere = np.argwhere(column)
            if argwhere.shape[0]==0:
                lowest = -1
            else:
                lowest = argwhere[-1,0]
            return lowest

        #self.reduced_boundary_matrix = self.get_reduced_boundary_matrix()

        persistence_representatives = []
        for j in range(len(self.filtration)):
            i_low = low(self.reduced_boundary_matrix[:,j])
            if i_low!=-1:
                birth_simplex, death_simplex = self.filtration[i_low], self.filtration[j]
                if (death_simplex.index - birth_simplex.index) > 1:
                    persistence_representative = PersistenceRepresentative(birth_simplex, death_simplex, np.zeros(1), np.zeros(1))
                    persistence_representatives.append(persistence_representative)

        return PersistenceDiagram(persistence_representatives)

    @property
    def harmonic_persistence_diagram(self):
        pass

class IndexFiltration:
    
    def __init__(self, cmplx):
        self.cmplx = cmplx

    def __call__(self, identity=False):
        
        if identity==False:
            filtered_cmplx = sorted(self.cmplx, key=lambda simplex: (simplex.index, simplex.vertices))
        else: # if identity - set index and time as they passed
            filtered_cmplx = self.cmplx
            for i, simplex in enumerate(filtered_cmplx):
                simplex.index = i

        for simplex in filtered_cmplx:
            simplex.time = simplex.index

        return FilteredComplex(filtered_cmplx)

class VietorisRipsFiltration:
    
    def __init__(self, X, distance_matrix=False):
        def pairwise_distances(X):
            return np.linalg.norm(X[:, None, :] - X[None, :, :], axis=-1)

        if (distance_matrix):
            self.X = X
        else:
            self.X = pairwise_distances(X)

        self.n_vertices = X.shape[0]

    def __call__(self):
        res = np.inf
        for pt in X:
            dst = 0
            for pt2 in X:
                dst = max(dst, np.linalg.norm(pt - pt2))
            res = min(res, dst)
        
        f = d.fill_rips(X, 2, res)
        filtered_cmplx = []
        print(len(f))
        sys.stdout.flush()
        for s in f:
            cur = Simplex(tuple(s))
            cur.time = s.data
            filtered_cmplx.append(cur)

        for i, simplex in enumerate(filtered_cmplx):
            simplex.index = i
            
        return FilteredComplex(filtered_cmplx, f)


In [10]:
# import numpy as np
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt


# set random state
rs = np.random.RandomState(42)

# generate data
# n_points = 30
# noise = rs.normal(1, 0.2) * 0.2
# X, _ = make_circles(n_samples=(n_points, 0), noise=noise, random_state=42)

def check_collision(current_circle, circles):
    """
        checks whether current_circle not intersects with circles
        returns: False if there is a collision, True otherwise
    """
    x, y, radius = current_circle
    if x-radius < 0 or x+radius > 1 or y-radius < 0 or y+radius > 1:
        return False
    return all(np.sqrt((x - xc)**2 + (y - yc)**2) >= (radius + rc) for xc, yc, rc in circles)

def in_circle(point, circle):
        """
            checks whether point is inside the circle
        """
        x, y = point
        xc, yc, r = circle
        return (x - xc) ** 2 + (y - yc) ** 2 < r ** 2


def generate_circles(num_circles,
                     r_boundaries,
                     x_boundaries,
                     y_boundaries,
                     num_attempts=1000,
                     seed=None):
    """
        Generates non-intersecting circles inside [0,1]x[0,1].
        num_circles - number of circles
        r_boundaries - circles radius boundaries
        x/y_boundaries - circles placement boundaries

        returns: list of non-intersecting circles
    """
    np.random.seed(seed)
    circles = []
    for _ in range(num_circles):
        for _ in range(num_attempts):
            radius = np.random.uniform(*(r_boundaries))
            x = np.random.uniform(x_boundaries[0] + radius, x_boundaries[1] - radius)
            y = np.random.uniform(y_boundaries[0] + radius, y_boundaries[1] - radius)
            new_circle = (x, y, radius)
            if check_collision((x, y, radius), circles):
                circles.append(new_circle)
                break
    return circles


def generate_example(num_points,
                     num_circles,
                     r_boundaries,
                     x_boundaries=(0, 1),
                     y_boundaries=(0, 1),
                     seed=None):
    """
    """
    np.random.seed(seed)
    points = np.random.rand(num_points, 2)
    circles = generate_circles(num_circles, r_boundaries, x_boundaries, y_boundaries, seed=seed)
    filtered_points = np.array([point for point in points if not any(in_circle(point, circle) for circle in circles)])
    return filtered_points, circles

X, _ = generate_example(num_circles=2, num_points=50, r_boundaries=(0.1, 0.15), x_boundaries=(0.5, 0.99))
print(len(X))
sys.stdout.flush()

# compute VR filtration of X
vr_filtration = VietorisRipsFiltration(X)
filtered_complex = vr_filtration()

print("@@@")
sys.stdout.flush()

# reduce the boundary matrix\
filtered_complex.get_reduced_boundary_matrix()

print("!!!")
sys.stdout.flush()
result = filtered_complex.spectra(128300)
print(result[0])

47
6746
(6746, 6746) 19376
Done
@@@
!!! <class 'scipy.sparse._lil.lil_matrix'>
m calculated <class 'scipy.sparse._lil.lil_matrix'> 6746
to get_persistence_diagram
[Diagram with 47 points, Diagram with 8 points, Diagram with 5303 points] [Diagram with 47 points, Diagram with 675 points, Diagram with 5303 points]
1 104.0 114.0
1 228.0 348.0
1 242.0 300.0
1 353.0 464.0
1 380.0 1275.0
1 391.0 446.0
1 499.0 1031.0
1 533.0 621.0
!!!
<class 'scipy.sparse._csr.csr_matrix'> (721, 721)
[ 8.88500041 10.45198926 10.86774028 11.7497642  12.34902373 12.62204015
 12.72611969 13.05492042 13.16218643 13.2684793 ]


In [19]:
import matplotlib.pyplot as plt

import numpy as np

def stack(X, idx):
    ret = np.empty((0, 2))
    for _id in idx:
        ret = np.vstack((ret, X[_id,:]))
    return ret

def harmonic_cycle0(m, eigenvector, simplices, aggregation=None):
    H = np.zeros((m, m))

    idx_i, idx_j = [], []
    for simplex in simplices:
        idx_i.append(simplex.vertices[0])
        idx_j.append(simplex.vertices[1])
    print(max(idx_i), max(idx_j))

    H[idx_i,idx_j] = np.abs(eigenvector)
    H = H + H.T # symmetrize

    if aggregation=="sum":
        H = H.sum(axis=0)
    elif aggregation=="max":
        H = H.max(axis=0)

    H = H / H.sum() # normalization

    return H

def plot(X, filtered_complex, eigenvalues, eigenvectors, harmonic_cycles, indices):
    def stack(idx):
        ret = np.empty((0, 2))
        for _id in idx:
            ret = np.vstack((ret, X[_id,:]))
        return ret

    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(14.25,4))

    for j in range(len(indices)):

        h = harmonic_cycles[j]
        index = indices[j]
        alphas = np.abs(eigenvectors[:,j]) ** 1.75 / np.max(np.abs(eigenvectors[:,j]) ** 1.75)

        # plot edges
        edges = filtered_complex.simplices_at_index[1]
        for k, edge in enumerate(edges):
            x_values = [X[edge.vertices[0],0], X[edge.vertices[1],0]]
            y_values = [X[edge.vertices[0],1], X[edge.vertices[1],1]]
            ax[j].set_title("$\lambda_{}$={:.4f}".format(j, eigenvalues[j]))
            ax[j].plot(x_values, y_values, "k-", alpha=0.15, linewidth=0.5)
            ax[j].plot(x_values, y_values, "r-", alpha=alphas[k])

        # plot triangles
        # triangles = filtered_complex.simplices_at_index[2]
        # for triangle in triangles:
        #     t = plt.Polygon(stack(triangle.vertices), color="red", alpha=0.025)
        #     ax[j].add_patch(t)

        if (j==0):
            ax[j].scatter(X[:,0], X[:,1], c=h, cmap="Reds", s=22.5)
        else:
            ax[j].scatter(X[:,0], X[:,1], c="k", alpha=0.5, s=10)

    plt.suptitle("Index: {:.0f}".format(index))
    plt.tight_layout()
    plt.show()


def plot2(X, edges_list, eigenvectors, harmonic_cycles, indices):
    def stack(idx):
        ret = np.empty((0, 2))
        for _id in idx:
            ret = np.vstack((ret, X[_id,:]))
        return ret

    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(14.25,4))

    for j, _ in enumerate(indices):

        edges = edges_list[j]
        eigenvector = np.abs(eigenvectors[j])
        h = harmonic_cycles[j]
        index = indices[j]

        alphas = eigenvector ** 1.75 / np.max(eigenvector ** 1.75)

        # plot edges
        for k, edge in enumerate(edges):
            x_values = [X[edge.vertices[0],0], X[edge.vertices[1],0]]
            y_values = [X[edge.vertices[0],1], X[edge.vertices[1],1]]
            ax[j].set_title("Index={}".format(index)) 
            ax[j].plot(x_values, y_values, "k-", alpha=0.15, linewidth=0.5)
            ax[j].plot(x_values, y_values, "r-", alpha=alphas[k])

        # plot triangles
        # triangles = filtered_complex.simplices_at_index[2]
        # for triangle in triangles:
        #     t = plt.Polygon(stack(triangle.vertices), color="red", alpha=0.025)
        #     ax[j].add_patch(t)

        if (j==0):
            ax[j].scatter(X[:,0], X[:,1], c=h, cmap="Reds", s=22.5)
        else:
            ax[j].scatter(X[:,0], X[:,1], c="k", alpha=0.5, s=10)

    plt.suptitle("Index: {:.0f}".format(index))
    plt.tight_layout()
    plt.show()

def plot_harmonic(X, filtered_complex):

    # get 1-dimensional homology classes
    representaives1 = filtered_complex.persistence_diagram.representatives_graded(k=1)
    n_representatives1 = len(representaives1)

    width = n_representatives1 * 4.25 + (n_representatives1-1) * 0.5

    fig, ax = plt.subplots(nrows=2, ncols=n_representatives1, figsize=(width,8.965))

    birth_row, death_row = 0, 1

    # plot points
    for row in range(2):
        for col in range(0, n_representatives1):
            ax[row,col].scatter(X[:,0], X[:,1], c="k", alpha=0.2, s=3)

    # representative columns
    for col in range(0, n_representatives1):

        repr_idx = col
        birth_index = representaives1[repr_idx].birth_simplex.index
        death_index = representaives1[repr_idx].death_simplex.index

        ax[birth_row,col].set_title("Optimal cycle at {}, [{}, {})".format(birth_index, birth_index, death_index))
        ax[death_row,col].set_title("Optimal volume cycle at {}, [{}, {})".format(death_index-1, birth_index, death_index))

        vertices_birth = filtered_complex.filtration_at_index(birth_index, k=0)
        edges_birth = filtered_complex.filtration_at_index(birth_index, k=1)
        alphas_birth = np.abs(representaives1[repr_idx].birth_harmonic_cycle)

        # aggregation
        m = len(vertices_birth)
        A = np.zeros((m, m))

        for vertex in vertices_birth:
            idx = vertex.vertices[0]
            x, y = X[idx,0], X[idx,1]
            ax[birth_row,col].annotate(idx, (x+0.02, y+0.01))

        for k, edge in enumerate(edges_birth):
            i, j = edge.vertices[0], edge.vertices[1]
            A[i,j] = alphas_birth[k]
            #print("BIRTH col={}, edge[{},{}]={}".format(col, i, j, alphas_birth[k]))

        # repr0_birth = A.sum(axis=0) # aggregate

        # mean aggregation
        sum = np.sum(A, axis=0)
        nz = np.count_nonzero(A, axis=0)
        repr0_birth = np.nan_to_num(sum / nz)
        repr0_birth = repr0_birth / repr0_birth.sum(axis=0) # normalize

        # print("VERTICES BIRTH", len(vertices_birth), vertices_birth)
        # print("EDGES / ALPHAS LEN", len(edges_birth), len(alphas_birth))
        # print("repr0_birth", repr0_birth)

        for k, edge in enumerate(edges_birth):
            x_values = [X[edge.vertices[0],0], X[edge.vertices[1],0]]
            y_values = [X[edge.vertices[0],1], X[edge.vertices[1],1]]
            ax[birth_row,col].plot(x_values, y_values, "k-", alpha=0.25, linewidth=0.33)
            ax[birth_row,col].plot(x_values, y_values, "r-", alpha=alphas_birth[k], linewidth=3)

        scaling_factor = 250
        ax[birth_row,col].scatter(X[:,0], X[:,1], c="k", alpha=1, s=scaling_factor*repr0_birth)

        vertices_death = filtered_complex.filtration_at_index(death_index-1, k=0)
        edges_death = filtered_complex.filtration_at_index(death_index-1, k=1)
        alphas_death = np.abs(representaives1[repr_idx].death_harmonic_cycle)

        # aggregation
        m = len(vertices_birth)
        A = np.zeros((m, m))

        for k, edge in enumerate(edges_death):
            i, j = edge.vertices[0], edge.vertices[1]
            A[i,j] = alphas_death[k]

        # mean aggregation
        sum = np.sum(A, axis=0)
        nz = np.count_nonzero(A, axis=0)
        repr0_death = np.nan_to_num(sum / nz)
        repr0_death = repr0_death / repr0_death.sum(axis=0) # normalize

        for k, edge in enumerate(edges_death):
            x_values = [X[edge.vertices[0],0], X[edge.vertices[1],0]]
            y_values = [X[edge.vertices[0],1], X[edge.vertices[1],1]]
            ax[death_row,col].plot(x_values, y_values, "k-", alpha=0.25, linewidth=0.33)
            ax[death_row,col].plot(x_values, y_values, "r-", alpha=alphas_death[k], linewidth=3)

        scaling_factor = 250
        ax[death_row,col].scatter(X[:,0], X[:,1], c="k", alpha=0.8, s=scaling_factor*repr0_death)

        # print("# EDGES BIRTH/DEATH", len(edges_birth), len(edges_death))
        # print(representaives1[repr_idx])

        # plot triangles
        triangles_birth = filtered_complex.filtration_at_index(birth_index, k=2)
        for triangle in triangles_birth:
            t = plt.Polygon(stack(X, triangle.vertices), color="red", alpha=0.03)
            ax[birth_row,col].add_patch(t)

        triangles_death = filtered_complex.filtration_at_index(death_index-1, k=2)
        for triangle in triangles_death:
            t = plt.Polygon(stack(X, triangle.vertices), color="red", alpha=0.03)
            ax[death_row,col].add_patch(t)

    # plt.suptitle("Index: {:.0f}".format(index))
    plt.tight_layout()
    plt.show()

In [6]:
def check_collision(current_circle, circles):
    """
        checks whether current_circle not intersects with circles
        returns: False if there is a collision, True otherwise
    """
    x, y, radius = current_circle
    if x-radius < 0 or x+radius > 1 or y-radius < 0 or y+radius > 1:
        return False
    return all(np.sqrt((x - xc)**2 + (y - yc)**2) >= (radius + rc) for xc, yc, rc in circles)

def in_circle(point, circle):
        """
            checks whether point is inside the circle
        """
        x, y = point
        xc, yc, r = circle
        return (x - xc) ** 2 + (y - yc) ** 2 < r ** 2


def generate_circles(num_circles,
                     r_boundaries,
                     x_boundaries,
                     y_boundaries,
                     num_attempts=1000,
                     seed=None):
    """
        Generates non-intersecting circles inside [0,1]x[0,1].
        num_circles - number of circles
        r_boundaries - circles radius boundaries
        x/y_boundaries - circles placement boundaries

        returns: list of non-intersecting circles
    """
    np.random.seed(seed)
    circles = []
    for _ in range(num_circles):
        for _ in range(num_attempts):
            radius = np.random.uniform(*(r_boundaries))
            x = np.random.uniform(x_boundaries[0] + radius, x_boundaries[1] - radius)
            y = np.random.uniform(y_boundaries[0] + radius, y_boundaries[1] - radius)
            new_circle = (x, y, radius)
            if check_collision((x, y, radius), circles):
                circles.append(new_circle)
                break
    return circles


def generate_example(num_points,
                     num_circles,
                     r_boundaries,
                     x_boundaries=(0, 1),
                     y_boundaries=(0, 1),
                     seed=None):
    """
    """
    np.random.seed(seed)
    points = np.random.rand(num_points, 2)
    circles = generate_circles(num_circles, r_boundaries, x_boundaries, y_boundaries, seed=seed)
    filtered_points = np.array([point for point in points if not any(in_circle(point, circle) for circle in circles)])
    return filtered_points, circles

In [81]:
%%time
import numpy as np
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt


# set random state
rs = np.random.RandomState(42)

# generate data
# n_points = 30
# noise = rs.normal(1, 0.2) * 0.2
# X, _ = make_circles(n_samples=(n_points, 0), noise=noise, random_state=42)

def check_collision(current_circle, circles):
    """
        checks whether current_circle not intersects with circles
        returns: False if there is a collision, True otherwise
    """
    x, y, radius = current_circle
    if x-radius < 0 or x+radius > 1 or y-radius < 0 or y+radius > 1:
        return False
    return all(np.sqrt((x - xc)**2 + (y - yc)**2) >= (radius + rc) for xc, yc, rc in circles)

def in_circle(point, circle):
        """
            checks whether point is inside the circle
        """
        x, y = point
        xc, yc, r = circle
        return (x - xc) ** 2 + (y - yc) ** 2 < r ** 2


def generate_circles(num_circles,
                     r_boundaries,
                     x_boundaries,
                     y_boundaries,
                     num_attempts=1000,
                     seed=None):
    """
        Generates non-intersecting circles inside [0,1]x[0,1].
        num_circles - number of circles
        r_boundaries - circles radius boundaries
        x/y_boundaries - circles placement boundaries

        returns: list of non-intersecting circles
    """
    np.random.seed(seed)
    circles = []
    for _ in range(num_circles):
        for _ in range(num_attempts):
            radius = np.random.uniform(*(r_boundaries))
            x = np.random.uniform(x_boundaries[0] + radius, x_boundaries[1] - radius)
            y = np.random.uniform(y_boundaries[0] + radius, y_boundaries[1] - radius)
            new_circle = (x, y, radius)
            if check_collision((x, y, radius), circles):
                circles.append(new_circle)
                break
    return circles


def generate_example(num_points,
                     num_circles,
                     r_boundaries,
                     x_boundaries=(0, 1),
                     y_boundaries=(0, 1),
                     seed=None):
    """
    """
    np.random.seed(seed)
    points = np.random.rand(num_points, 2)
    circles = generate_circles(num_circles, r_boundaries, x_boundaries, y_boundaries, seed=seed)
    filtered_points = np.array([point for point in points if not any(in_circle(point, circle) for circle in circles)])
    return filtered_points, circles

X, _ = generate_example(num_circles=2, num_points=300, r_boundaries=(0.1, 0.15), x_boundaries=(0.5, 0.99))
print("Total pts:", len(X))

# compute VR filtration of X
vr_filtration = VietorisRipsFiltration(X)
filtered_complex = vr_filtration()

print("@@@")

# reduce the boundary matrix
filtered_complex.get_reduced_boundary_matrix()

print("!!!")

# print diagram
# print(filtered_complex.persistence_diagram.as_numpy(index=True))

threshold = 1e-6
print("LEN", len(filtered_complex.persistence_diagram.elements))
##print(len(filtered_complex.persistence_diagram.elements))
ph_lens = []
for i, eq in enumerate(filtered_complex.persistence_diagram.elements):
    ph_lens.append((eq.death_simplex.index - eq.birth_simplex.index, i, eq.death_simplex.index))
print(sorted(ph_lens, reverse=True))
ok_ids = list(map(lambda x: x[1], sorted(ph_lens, reverse=True)[:5]))
print(ok_ids)

# for each H_1 homology class
for i, eq in enumerate(filtered_complex.persistence_diagram.elements):
    if i not in ok_ids:
        continue
    if eq.birth_simplex.dim==1:

        # birth and death indices
        birth_index = eq.birth_simplex.index
        death_index = eq.death_simplex.index

        # eigenvectors, eigenvalues and edges at birth-1, birth, death-1, and death
        eigenvalues_v, eigenvectors_v = filtered_complex.spectra(birth_index-1)
        eigenvalues_x, eigenvectors_x = filtered_complex.spectra(birth_index)
        eigenvalues_y, eigenvectors_y = filtered_complex.spectra(death_index-1)
        eigenvalues_z, eigenvectors_z = filtered_complex.spectra(death_index)
        print(eigenvectors_v.shape, eigenvectors_x.shape, eigenvectors_y.shape, eigenvectors_z.shape)

        # eigenvectors corresponding to near-zero eigenvalues
        eigenvectors_v = eigenvectors_v[:,np.isclose(eigenvalues_v, np.zeros_like(eigenvalues_v), threshold)]
        eigenvectors_x = eigenvectors_x[:,np.isclose(eigenvalues_x, np.zeros_like(eigenvalues_x), threshold)]
        eigenvectors_y = eigenvectors_y[:,np.isclose(eigenvalues_y, np.zeros_like(eigenvalues_y), threshold)]
        eigenvectors_z = eigenvectors_z[:,np.isclose(eigenvalues_z, np.zeros_like(eigenvalues_z), threshold)]
        print(eigenvectors_v.shape, eigenvectors_x.shape, eigenvectors_y.shape, eigenvectors_z.shape)

        # print(i+1, "BIRTH, DEATH", birth_index, death_index)

        # print("V SHAPE", eigenvectors_v.shape)
        # print("X SHAPE", eigenvectors_x.shape)
        # print("Y SHAPE", eigenvectors_y.shape)
        # print("Z SHAPE", eigenvectors_z.shape)

        # optimal cycle
        if eigenvectors_v.shape[1]!=0:
            lambdas = np.zeros(eigenvectors_x.shape[1])
            for j in range(eigenvectors_x.shape[1]):
                Hvx_j = np.concatenate((eigenvectors_v, eigenvectors_x[:-1,j][:,np.newaxis]), axis=1)
                # print("(V | x_j)", Hvx_j.shape)
                _, singular_values, _ = np.linalg.svd(Hvx_j)
                lambdas[j] = np.min(singular_values)
            idx = np.argmax(lambdas)
            optimal_cycle = eigenvectors_x[:,idx]
        else:
            optimal_cycle = eigenvectors_x[:,0]

        # cycle of optimal volume
        if eigenvectors_z.shape[1]!=0:
            lambdas = np.zeros(eigenvectors_y.shape[1])
            for j in range(eigenvectors_y.shape[1]):
                Hzy_j = np.concatenate((eigenvectors_z, eigenvectors_y[:,j][:,np.newaxis]), axis=1)
                # print("(Z | y_j)", Hzy_j.shape)
                _, singular_values, _ = np.linalg.svd(Hzy_j)
                lambdas[j] = np.min(singular_values)
            idx = np.argmax(lambdas)
            optimal_volume_cycle = eigenvectors_y[:,idx]
        else:
            optimal_volume_cycle = eigenvectors_y[:,0]

        # print("OPTIMAL CYCLE", i)
        # print(optimal_cycle)
        # print("OPTIMAL VOLUME CYCLE", i)
        # print(optimal_volume_cycle)

        # update persistence diagram
        filtered_complex.persistence_diagram.elements[i].birth_harmonic_cycle = optimal_cycle
        filtered_complex.persistence_diagram.elements[i].death_harmonic_cycle = optimal_volume_cycle

# plot harmonic diagram
print("!!!")
sys.stdout.flush()
#plot_harmonic(X, filtered_complex)


Total pts: 256
1117607
(1117607, 1117607) 3329554
Done
@@@
!!! <class 'scipy.sparse._lil.lil_matrix'>
m calculated <class 'scipy.sparse._lil.lil_matrix'> 1117607
to get_persistence_diagram
[Diagram with 256 points, Diagram with 55 points, Diagram with 1072608 points] [Diagram with 256 points, Diagram with 22244 points, Diagram with 1072608 points]
1 464.0 648.0
1 541.0 633.0
1 547.0 727.0
1 861.0 1190.0
1 862.0 1002.0
1 871.0 906.0
1 879.0 901.0
1 894.0 1767.0
1 924.0 1642.0
1 982.0 1141.0
1 1003.0 1036.0
1 1006.0 1286.0
1 1007.0 4883.0
1 1021.0 1204.0
1 1107.0 1696.0
1 1116.0 1371.0
1 1252.0 4635.0
1 1318.0 3101.0
1 1324.0 3611.0
1 1341.0 1460.0
1 1344.0 1981.0
1 1350.0 64593.0
1 1365.0 1415.0
1 1388.0 1403.0
1 1433.0 2003.0
1 1486.0 2077.0
1 1493.0 1607.0
1 1514.0 3682.0
1 1525.0 1536.0
1 1579.0 2409.0
1 1616.0 3049.0
1 1623.0 3818.0
1 1649.0 2502.0
1 1755.0 3462.0
1 1772.0 3309.0
1 1810.0 1829.0
1 1982.0 2035.0
1 1993.0 2012.0
1 2040.0 2253.0
1 2051.0 2285.0
1 2177.0 2317.0
1 2197.0

In [29]:
def gen(X):
    N = len(X)
    print("Total pts:", len(X))

    # compute VR filtration of X
    vr_filtration = VietorisRipsFiltration(X)
    filtered_complex = vr_filtration()

    print("@@@")

    # reduce the boundary matrix
    filtered_complex.get_reduced_boundary_matrix()

    print("!!!")

    # print diagram
    # print(filtered_complex.persistence_diagram.as_numpy(index=True))

    threshold = 1e-6
    print("LEN", len(filtered_complex.persistence_diagram.elements))
    ##print(len(filtered_complex.persistence_diagram.elements))
    ph_lens = []
    for i, eq in enumerate(filtered_complex.persistence_diagram.elements):
        ph_lens.append((eq.death_simplex.index - eq.birth_simplex.index, i, eq.death_simplex.index))
    print(sorted(ph_lens, reverse=True))
    ok_ids = list(map(lambda x: x[1], sorted(ph_lens, reverse=True)[:min(5, len(ph_lens))]))
    print(ok_ids)

    # for each H_1 homology class
    a, b = [], []
    for i, eq in enumerate(filtered_complex.persistence_diagram.elements):
        if i not in ok_ids:
            continue
        if eq.birth_simplex.dim==1:

            # birth and death indices
            birth_index = eq.birth_simplex.index
            death_index = eq.death_simplex.index

            # eigenvectors, eigenvalues and edges at birth-1, birth, death-1, and death
            eigenvalues_v, eigenvectors_v = filtered_complex.spectra(birth_index-1)
            eigenvalues_x, eigenvectors_x = filtered_complex.spectra(birth_index)
            eigenvalues_y, eigenvectors_y = filtered_complex.spectra(death_index-1)
            eigenvalues_z, eigenvectors_z = filtered_complex.spectra(death_index)
            print(eigenvectors_v.shape, eigenvectors_x.shape, eigenvectors_y.shape, eigenvectors_z.shape)

            # eigenvectors corresponding to near-zero eigenvalues
            eigenvectors_v = eigenvectors_v[:,np.isclose(eigenvalues_v, np.zeros_like(eigenvalues_v), threshold)]
            eigenvectors_x = eigenvectors_x[:,np.isclose(eigenvalues_x, np.zeros_like(eigenvalues_x), threshold)]
            eigenvectors_y = eigenvectors_y[:,np.isclose(eigenvalues_y, np.zeros_like(eigenvalues_y), threshold)]
            eigenvectors_z = eigenvectors_z[:,np.isclose(eigenvalues_z, np.zeros_like(eigenvalues_z), threshold)]
            print(eigenvectors_v.shape, eigenvectors_x.shape, eigenvectors_y.shape, eigenvectors_z.shape)

            # print(i+1, "BIRTH, DEATH", birth_index, death_index)

            # print("V SHAPE", eigenvectors_v.shape)
            # print("X SHAPE", eigenvectors_x.shape)
            # print("Y SHAPE", eigenvectors_y.shape)
            # print("Z SHAPE", eigenvectors_z.shape)

            # optimal cycle
            if eigenvectors_v.shape[1]!=0:
                lambdas = np.zeros(eigenvectors_x.shape[1])
                for j in range(eigenvectors_x.shape[1]):
                    Hvx_j = np.concatenate((eigenvectors_v, eigenvectors_x[:-1,j][:,np.newaxis]), axis=1)
                    # print("(V | x_j)", Hvx_j.shape)
                    _, singular_values, _ = np.linalg.svd(Hvx_j)
                    lambdas[j] = np.min(singular_values)
                idx = np.argmax(lambdas)
                optimal_cycle = eigenvectors_x[:,idx]
            else:
                optimal_cycle = eigenvectors_x[:,0]

            # cycle of optimal volume
            if eigenvectors_z.shape[1]!=0:
                lambdas = np.zeros(eigenvectors_y.shape[1])
                for j in range(eigenvectors_y.shape[1]):
                    Hzy_j = np.concatenate((eigenvectors_z, eigenvectors_y[:,j][:,np.newaxis]), axis=1)
                    # print("(Z | y_j)", Hzy_j.shape)
                    _, singular_values, _ = np.linalg.svd(Hzy_j)
                    lambdas[j] = np.min(singular_values)
                idx = np.argmax(lambdas)
                optimal_volume_cycle = eigenvectors_y[:,idx]
            else:
                optimal_volume_cycle = eigenvectors_y[:,0]

            # print("OPTIMAL CYCLE", i)
            # print(optimal_cycle)
            # print("OPTIMAL VOLUME CYCLE", i)
            # print(optimal_volume_cycle)

            # update persistence diagram
            filtered_complex.persistence_diagram.elements[i].birth_harmonic_cycle = optimal_cycle
            filtered_complex.persistence_diagram.elements[i].death_harmonic_cycle = optimal_volume_cycle
            a.append([birth_index, death_index])
            b.append(harmonic_cycle0(N, optimal_cycle, filtered_complex.simplices_at_index[1][:len(optimal_cycle)], aggregation="max"))
            print(len(optimal_cycle))
    return np.array(a), np.array(b)

In [30]:
X, _ = generate_example(num_circles=2, num_points=100, r_boundaries=(0.1, 0.15), x_boundaries=(0.5, 0.99))
len(X)

86

In [31]:
a, b = gen(X)

Total pts: 86
52499
(52499, 52499) 154501
Done
@@@
!!! <class 'scipy.sparse._lil.lil_matrix'>
m calculated <class 'scipy.sparse._lil.lil_matrix'> 52499
to get_persistence_diagram
[Diagram with 86 points, Diagram with 17 points, Diagram with 47022 points] [Diagram with 86 points, Diagram with 2653 points, Diagram with 47022 points]
1 142.0 146.0
1 316.0 338.0
1 329.0 502.0
1 334.0 376.0
1 371.0 425.0
1 493.0 935.0
1 600.0 655.0
1 631.0 775.0
1 682.0 688.0
1 732.0 837.0
1 764.0 2046.0
1 772.0 781.0
1 1150.0 1488.0
1 1175.0 3549.0
1 1304.0 3301.0
1 1315.0 1385.0
1 2841.0 3152.0
!!!
LEN 17
[(2374, 13, 3549), (1997, 14, 3301), (1282, 10, 2046), (442, 5, 935), (338, 12, 1488), (311, 16, 3152), (173, 2, 502), (144, 7, 775), (105, 9, 837), (70, 15, 1385), (55, 6, 655), (54, 4, 425), (42, 3, 376), (22, 1, 338), (9, 11, 781), (6, 8, 688), (4, 0, 146)]
[13, 14, 10, 5, 12]
<class 'scipy.sparse._csr.csr_matrix'> (201, 201)
<class 'scipy.sparse._csr.csr_matrix'> (202, 202)
<class 'scipy.sparse._csr.

In [33]:
a.shape, b.shape

((5, 2), (5, 86))

In [34]:
a, b

(array([[ 493,  935],
        [ 764, 2046],
        [1150, 1488],
        [1175, 3549],
        [1304, 3301]]),
 array([[1.07437938e-17, 1.97330024e-02, 4.86090167e-02, 1.86231267e-17,
         3.56084368e-19, 1.60741529e-02, 1.72706921e-02, 2.60517017e-17,
         1.31920989e-16, 4.72347804e-17, 1.63625225e-16, 2.54209721e-02,
         8.58246509e-18, 3.04669654e-19, 7.03726867e-17, 1.31920989e-16,
         3.75063568e-02, 1.85539949e-02, 1.75634485e-02, 2.78849716e-02,
         0.00000000e+00, 5.07639405e-17, 1.73577045e-02, 6.05798359e-17,
         3.62846336e-17, 3.95273408e-18, 5.16789917e-18, 5.06925702e-17,
         2.51360917e-04, 2.44316186e-02, 8.31887496e-17, 4.27175010e-17,
         1.81464253e-02, 1.06073232e-16, 7.30272724e-17, 7.37919523e-17,
         4.23532349e-17, 7.19612171e-04, 2.76827336e-02, 1.12519070e-01,
         3.04669654e-19, 5.50229978e-17, 1.51909731e-16, 2.69854564e-03,
         3.70688735e-02, 1.46269939e-16, 1.21795041e-16, 3.56084368e-19,
         4.0

In [18]:
X = np.random.normal(size=(50, 2))
X.shape

(50, 2)

In [19]:
vr_filtration = VietorisRipsFiltration(X)
filtered_complex = vr_filtration()

12212
(12212, 12212) 35504
Done


In [20]:
filtered_complex.get_reduced_boundary_matrix()

!!! <class 'scipy.sparse._lil.lil_matrix'>
m calculated <class 'scipy.sparse._lil.lil_matrix'> 12212
to get_persistence_diagram
[Diagram with 50 points, Diagram with 9 points, Diagram with 10247 points] [Diagram with 50 points, Diagram with 933 points, Diagram with 10247 points]
1 100.0 113.0
1 141.0 255.0
1 156.0 161.0
1 169.0 825.0
1 174.0 192.0
1 177.0 211.0
1 184.0 418.0
1 537.0 743.0
1 604.0 620.0


<12212x12212 sparse matrix of type '<class 'numpy.int64'>'
	with 2937 stored elements in List of Lists format>

In [21]:
threshold = 1e-6
print("LEN", len(filtered_complex.persistence_diagram.elements))
##print(len(filtered_complex.persistence_diagram.elements))
ph_lens = []
for i, eq in enumerate(filtered_complex.persistence_diagram.elements):
    ph_lens.append((eq.death_simplex.index - eq.birth_simplex.index, i, eq.death_simplex.index))
print(sorted(ph_lens, reverse=True))
ok_ids = list(map(lambda x: x[1], sorted(ph_lens, reverse=True)[:5]))
print(ok_ids)

LEN 9
[(656, 3, 825), (234, 6, 418), (206, 7, 743), (114, 1, 255), (34, 5, 211), (18, 4, 192), (16, 8, 620), (13, 0, 113), (5, 2, 161)]
[3, 6, 7, 1, 5]


In [23]:
res1 = []
for i, eq in enumerate(filtered_complex.persistence_diagram.elements):
    if eq.birth_simplex.dim==1:

        # birth and death indices
        birth_index = eq.birth_simplex.index
        death_index = eq.death_simplex.index

        # eigenvectors, eigenvalues and edges at birth-1, birth, death-1, and death
        eigenvalues_v, eigenvectors_v = filtered_complex.spectra(birth_index-1)
        eigenvalues_x, eigenvectors_x = filtered_complex.spectra(birth_index)
        eigenvalues_y, eigenvectors_y = filtered_complex.spectra(death_index-1)
        eigenvalues_z, eigenvectors_z = filtered_complex.spectra(death_index)
        print(eigenvectors_v.shape, eigenvectors_x.shape, eigenvectors_y.shape, eigenvectors_z.shape)

        # eigenvectors corresponding to near-zero eigenvalues
        eigenvectors_v = eigenvectors_v[:,np.isclose(eigenvalues_v, np.zeros_like(eigenvalues_v), threshold)]
        eigenvectors_x = eigenvectors_x[:,np.isclose(eigenvalues_x, np.zeros_like(eigenvalues_x), threshold)]
        eigenvectors_y = eigenvectors_y[:,np.isclose(eigenvalues_y, np.zeros_like(eigenvalues_y), threshold)]
        eigenvectors_z = eigenvectors_z[:,np.isclose(eigenvalues_z, np.zeros_like(eigenvalues_z), threshold)]
        print(eigenvectors_v.shape, eigenvectors_x.shape, eigenvectors_y.shape, eigenvectors_z.shape)

        # print(i+1, "BIRTH, DEATH", birth_index, death_index)

        # print("V SHAPE", eigenvectors_v.shape)
        # print("X SHAPE", eigenvectors_x.shape)
        # print("Y SHAPE", eigenvectors_y.shape)
        # print("Z SHAPE", eigenvectors_z.shape)

        # optimal cycle
        if eigenvectors_v.shape[1]!=0:
            lambdas = np.zeros(eigenvectors_x.shape[1])
            for j in range(eigenvectors_x.shape[1]):
                Hvx_j = np.concatenate((eigenvectors_v, eigenvectors_x[:-1,j][:,np.newaxis]), axis=1)
                # print("(V | x_j)", Hvx_j.shape)
                _, singular_values, _ = np.linalg.svd(Hvx_j)
                lambdas[j] = np.min(singular_values)
            idx = np.argmax(lambdas)
            optimal_cycle = eigenvectors_x[:,idx]
        else:
            optimal_cycle = eigenvectors_x[:,0]

        # cycle of optimal volume
        if eigenvectors_z.shape[1]!=0:
            lambdas = np.zeros(eigenvectors_y.shape[1])
            for j in range(eigenvectors_y.shape[1]):
                Hzy_j = np.concatenate((eigenvectors_z, eigenvectors_y[:,j][:,np.newaxis]), axis=1)
                # print("(Z | y_j)", Hzy_j.shape)
                _, singular_values, _ = np.linalg.svd(Hzy_j)
                lambdas[j] = np.min(singular_values)
            idx = np.argmax(lambdas)
            optimal_volume_cycle = eigenvectors_y[:,idx]
        else:
            optimal_volume_cycle = eigenvectors_y[:,0]

        # print("OPTIMAL CYCLE", i)
        # print(optimal_cycle)
        # print("OPTIMAL VOLUME CYCLE", i)
        # print(optimal_volume_cycle)

        # update persistence diagram
        filtered_complex.persistence_diagram.elements[i].birth_harmonic_cycle = optimal_cycle
        filtered_complex.persistence_diagram.elements[i].death_harmonic_cycle = optimal_volume_cycle
        res1.append()


<class 'scipy.sparse._csr.csr_matrix'> (42, 42)
<class 'scipy.sparse._csr.csr_matrix'> (43, 43)
<class 'scipy.sparse._csr.csr_matrix'> (49, 49)
<class 'scipy.sparse._csr.csr_matrix'> (49, 49)
(42, 10) (43, 10) (49, 10) (49, 10)
(42, 0) (43, 1) (49, 1) (49, 0)
<class 'scipy.sparse._csr.csr_matrix'> (63, 63)
<class 'scipy.sparse._csr.csr_matrix'> (64, 64)
<class 'scipy.sparse._csr.csr_matrix'> (116, 116)
<class 'scipy.sparse._csr.csr_matrix'> (116, 116)
(63, 10) (64, 10) (116, 10) (116, 10)
(63, 0) (64, 1) (116, 3) (116, 2)
<class 'scipy.sparse._csr.csr_matrix'> (72, 72)
<class 'scipy.sparse._csr.csr_matrix'> (73, 73)
<class 'scipy.sparse._csr.csr_matrix'> (75, 75)
<class 'scipy.sparse._csr.csr_matrix'> (75, 75)
(72, 10) (73, 10) (75, 10) (75, 10)
(72, 1) (73, 2) (75, 2) (75, 1)
<class 'scipy.sparse._csr.csr_matrix'> (78, 78)
<class 'scipy.sparse._csr.csr_matrix'> (79, 79)
<class 'scipy.sparse._csr.csr_matrix'> (250, 250)
<class 'scipy.sparse._csr.csr_matrix'> (250, 250)
(78, 10) (79, 10

In [24]:
filtered_complex.persistence_diagram.elements[0]

PersistenceRepresentative(birth_simplex=(9, 11), death_simplex=(9, 11, 36), birth_harmonic_cycle=array([-9.62828272e-18, -3.44174967e-17,  4.09344642e-17,  5.74425734e-17,
       -1.64286038e-16, -5.00000000e-01, -1.82272981e-16,  6.05710298e-17,
       -2.23027681e-16,  6.24575009e-17,  3.33183606e-16, -2.72124455e-17,
        4.30142580e-17,  9.28967276e-17,  1.55164676e-17, -8.73433477e-18,
       -1.26221228e-16,  5.00000000e-01, -2.48638875e-16, -7.46470640e-17,
        3.27653232e-16,  8.29166244e-17,  2.35959157e-16,  1.56699188e-16,
       -6.20512160e-17, -6.81605434e-19,  4.16287415e-17,  3.52628763e-17,
        4.61642669e-17, -1.91830753e-17,  3.96276498e-17, -3.58788032e-16,
       -9.88792381e-17,  1.89687333e-17,  2.07787346e-16, -1.47451495e-16,
        4.14504043e-17, -3.99426857e-17,  2.28983499e-16, -6.66411642e-17,
        5.00000000e-01, -1.19695920e-16, -5.00000000e-01]), death_harmonic_cycle=array([ 1.83281256e-17, -1.85766143e-16, -8.29751487e-18, -4.89590085e-1

In [44]:
f = d.fill_rips(X, 2, res)
m = d.homology_persistence(f)
for i,c in enumerate(m):
    if c:
        print(i, c)
        for t in c:
            print(t.index)
        break

29 1*6 + 1*20
6
20


In [36]:
for i, s in enumerate(f):
    print(list(s), s.data)
    if i > 50:
        break

[0] 0.0
[1] 0.0
[2] 0.0
[3] 0.0
[4] 0.0
[5] 0.0
[6] 0.0
[7] 0.0
[8] 0.0
[9] 0.0
[10] 0.0
[11] 0.0
[12] 0.0
[13] 0.0
[14] 0.0
[15] 0.0
[16] 0.0
[17] 0.0
[18] 0.0
[19] 0.0
[20] 0.0
[21] 0.0
[22] 0.0
[23] 0.0
[24] 0.0
[25] 0.0
[26] 0.0
[27] 0.0
[28] 0.0
[29] 0.0
[30] 0.0
[31] 0.0
[32] 0.0
[33] 0.0
[34] 0.0
[35] 0.0
[16, 30] 0.025616537779569626
[2, 29] 0.03701687231659889
[0, 34] 0.04234061390161514
[4, 20] 0.04704831913113594
[2, 35] 0.04920085892081261
[21, 28] 0.052800484001636505
[8, 11] 0.058194153010845184
[12, 23] 0.06830205023288727
[10, 18] 0.06883402168750763
[14, 30] 0.07068460434675217
[29, 35] 0.0710279643535614
[2, 29, 35] 0.0710279643535614
[3, 23] 0.07118809223175049
[14, 16] 0.07547935843467712
[14, 16, 30] 0.07547935843467712
[25, 26] 0.07817026227712631


In [14]:
simplices = [([2], 4), ([1,2], 5), ([0,2], 6),
             ([0], 1),   ([1], 2), ([0,1], 3)]
f = d.Filtration()
for vertices, time in simplices:
    f.append(d.Simplex(vertices, time))
f.sort()
m = d.homology_persistence(f)
dgms = d.init_diagrams(m, f)

In [17]:
for i, s in enumerate(f):
    print(i, s)

0 <0> 1
1 <1> 2
2 <0,1> 3
3 <2> 4
4 <1,2> 5
5 <0,2> 6


In [20]:
for i, dgm in enumerate(dgms):
    for pt in dgm:
        print(i, pt.birth, pt.death, pt.data)

0 1.0 inf 0
0 2.0 3.0 1
0 4.0 5.0 3
1 6.0 inf 5
